In [ ]:
import os
from urllib.request import urlopen
import ssl, certifi
import pandas as pd
import plotly.express as px

os.makedirs("../data", exist_ok=True)

ctx = ssl.create_default_context(cafile=certifi.where())

urls = {
    "unvotes":    "https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2021/2021-03-23/unvotes.csv",
    "roll_calls": "https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2021/2021-03-23/roll_calls.csv",
    "issues":     "https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2021/2021-03-23/issues.csv",
}

for name, url in urls.items():
    with urlopen(url, context=ctx) as f:
        df = pd.read_csv(f)
    out_path = f"../data/{name}.csv"
    df.to_csv(out_path, index=False)
    print(f"✅ Saved to {out_path}")

✅ Saved to ../data/unvotes.csv
✅ Saved to ../data/roll_calls.csv
✅ Saved to ../data/issues.csv


In [299]:
unvotes    = pd.read_csv('../data/unvotes.csv')
roll_calls = pd.read_csv('../data/roll_calls.csv')
issues = pd.read_csv('../data/issues.csv')

print(unvotes.head())
print(roll_calls.head())
print(issues.head())

   rcid             country country_code vote
0     3       United States           US  yes
1     3              Canada           CA   no
2     3                Cuba           CU  yes
3     3               Haiti           HT  yes
4     3  Dominican Republic           DO  yes
   rcid  session  importantvote        date    unres  amend  para  \
0     3        1            0.0  1946-01-01   R/1/66    1.0   0.0   
1     4        1            0.0  1946-01-02   R/1/79    0.0   0.0   
2     5        1            0.0  1946-01-04   R/1/98    0.0   0.0   
3     6        1            0.0  1946-01-04  R/1/107    0.0   0.0   
4     7        1            0.0  1946-01-02  R/1/295    1.0   0.0   

                            short  \
0  AMENDMENTS, RULES OF PROCEDURE   
1      SECURITY COUNCIL ELECTIONS   
2                VOTING PROCEDURE   
3     DECLARATION OF HUMAN RIGHTS   
4      GENERAL ASSEMBLY ELECTIONS   

                                               descr  
0  TO ADOPT A CUBAN AMENDMENT T

## Ultimate outer merge

In [300]:
import pandas as pd

# 1. Load the three CSV files into pandas DataFrames
unvotes    = pd.read_csv('../data/unvotes.csv')
roll_calls = pd.read_csv('../data/roll_calls.csv')
issues = pd.read_csv('../data/issues.csv')
# 2. Perform the outer merge on the three DataFrames using 'rcid' as the key
merged_df = roll_calls.merge(unvotes, on="rcid", how="outer").merge(issues, on="rcid", how="outer")

# 3. Reset index if you want to drop the old index column and get a clean index
merged_df = merged_df.reset_index(drop=True)

# 4. Save the merged DataFrame to a local folder as a CSV file
merged_df.to_csv("merged_data.csv", index=False)

# Optional: You can save it as an Excel or JSON if needed:
# merged_df.to_excel("path/to/save/merged_data.xlsx", index=False)
# merged_df.to_json("path/to/save/merged_data.json", orient="records", lines=True)

print("Merge completed and saved to local folder.")


Merge completed and saved to local folder.


In [301]:
merged_df.columns.unique()

#prove outer merge is completed
merged_df.shape


(1123334, 14)

## Question: "How do different regions vote on the key issue (Human Rights) in the UN, and have there been any noticeable shifts in voting behavior over time?"

### Show not all countries have country_code

In [302]:
# Filter rows where country_code is missing (NaN)
missing_country_code = merged_df[merged_df["country_code"].isna()]

# Get unique countries where country_code is missing
unique_countries_with_missing_code = missing_country_code['country'].unique()

# Display the result
print(unique_countries_with_missing_code)

['Yemen Arab Republic' 'Zanzibar' 'Federal Republic of Germany' 'Namibia']


Notes: 
Remove session cause each session refers to one year, 1 = 1946
Remove para. Because it is only on a paragraph and not a resolution. And it stops at 1985, which falls out of the date region we are trying to visualise
Same for amend
Remove unres. Because its a unique key as RCID and less sophisticated
Remove describe. Because every single line is unique and its very hard to filter out the info

Convert date to datetime. Then, extract the year from it\
The reason why we remove session and kept year is cause we can visualise it better
Countr_code. Not all countries have country_code hence there will be NA values. So we drop it 

Dont need short name as it is exactly the same as issue 


In [303]:
print(merged_df['short_name'].unique())
print(merged_df['issue'].unique())

[nan 'hr' 'ec' 'co' 'me' 'di' 'nu']
[nan 'Human rights' 'Economic development' 'Colonialism'
 'Palestinian conflict' 'Arms control and disarmament'
 'Nuclear weapons and nuclear material']


In [304]:
merged_df.columns.unique()

Index(['rcid', 'session', 'importantvote', 'date', 'unres', 'amend', 'para',
       'short', 'descr', 'country', 'country_code', 'vote', 'short_name',
       'issue'],
      dtype='object')

In [305]:
#Pick only the columns we need
final_df = merged_df[['rcid', 'date', 'country', 'vote', 'issue', 'importantvote', 'short']].copy()
print(final_df.head())

   rcid        date             country vote issue  importantvote  \
0     3  1946-01-01       United States  yes   NaN            0.0   
1     3  1946-01-01              Canada   no   NaN            0.0   
2     3  1946-01-01                Cuba  yes   NaN            0.0   
3     3  1946-01-01               Haiti  yes   NaN            0.0   
4     3  1946-01-01  Dominican Republic  yes   NaN            0.0   

                            short  
0  AMENDMENTS, RULES OF PROCEDURE  
1  AMENDMENTS, RULES OF PROCEDURE  
2  AMENDMENTS, RULES OF PROCEDURE  
3  AMENDMENTS, RULES OF PROCEDURE  
4  AMENDMENTS, RULES OF PROCEDURE  


In [306]:
print(final_df['issue'].nunique())
print(final_df['short'].nunique())

#therefore both of them are different and we cant merge inner 


6
2020


In [307]:
# 3. Reset index if you want to drop the old index column and get a clean index
final_df = final_df.reset_index(drop=True)

# 4. Save the merged DataFrame to a local folder as a CSV file
final_df.to_csv("final_merged_data.csv", index=False)

### Extract only year for date (using datetime)

In [308]:
final_df['date'] = pd.to_datetime(final_df['date']).dt.year
print(final_df.head())

   rcid  date             country vote issue  importantvote  \
0     3  1946       United States  yes   NaN            0.0   
1     3  1946              Canada   no   NaN            0.0   
2     3  1946                Cuba  yes   NaN            0.0   
3     3  1946               Haiti  yes   NaN            0.0   
4     3  1946  Dominican Republic  yes   NaN            0.0   

                            short  
0  AMENDMENTS, RULES OF PROCEDURE  
1  AMENDMENTS, RULES OF PROCEDURE  
2  AMENDMENTS, RULES OF PROCEDURE  
3  AMENDMENTS, RULES OF PROCEDURE  
4  AMENDMENTS, RULES OF PROCEDURE  


### Which issues are deemed more important (based on importantvote)?

In [316]:
usa_df = final_df.copy() 
usa_df = usa_df[['country', 'issue', 'importantvote', 'vote']]
usa_df = usa_df.dropna(subset=['issue', 'importantvote']).reset_index(drop=True)
print(usa_df.head())

              country         issue  importantvote     vote
0       United States  Human rights            0.0       no
1              Canada  Human rights            0.0       no
2                Cuba  Human rights            0.0      yes
3  Dominican Republic  Human rights            0.0  abstain
4              Mexico  Human rights            0.0      yes


### Check the different types of important vote

In [317]:
print(usa_df['importantvote'].unique())
print(usa_df['issue'].unique())
print(usa_df['vote'].unique())

[0. 1.]
['Human rights' 'Economic development' 'Colonialism'
 'Palestinian conflict' 'Arms control and disarmament'
 'Nuclear weapons and nuclear material']
['no' 'yes' 'abstain']


### Sum up importantvotes 

In [ ]:
usa_df['importance'] = usa_df.groupby('issue')['importantvote'].transform('sum')
print(usa_df.head())


              country         issue  importantvote     vote  importance
0       United States  Human rights            0.0       no     28844.0
1              Canada  Human rights            0.0       no     28844.0
2                Cuba  Human rights            0.0      yes     28844.0
3  Dominican Republic  Human rights            0.0  abstain     28844.0
4              Mexico  Human rights            0.0      yes     28844.0


### Plotting of graph

In [327]:
# Group by 'issue' and calculate the average importance for each issue
issue_importance = usa_df.groupby('issue')['importance'].mean().reset_index()

# Create a bar plot
fig = px.bar(
    issue_importance,
    x='issue',  # Issues on the x-axis
    y='importance',  # Average importance on the y-axis
    title="Average Importance of Issues in UN Votes",
    labels={"importance": "Average Importance", "issue": "Issue"},
    color='importance',  # Color by importance for better visualization
    color_continuous_scale="Viridis"  # Choose color scale
)

fig.update_layout(
    xaxis_title="Issue",
    yaxis_title="Average Importance"
)

# Show the plot
fig.show()


## Bryan's Graph